In [5]:
import pandas as pd 
import numpy as np 
import os
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime
from dateutil.relativedelta import relativedelta
import itertools

pd.set_option('display.max_columns', None)

# declaramos las credenciales al entorno de desarollo
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "..\\..\\bd\\0_docs\\datahub-deacero-adojeda.json"

# Crear date_today como el primer día del mes actual
now = datetime.now() - relativedelta(days= 4)

date_today = datetime(now.year, now.month, 1)

# Crear date_last_6m como seis meses antes de date_today
date_last_6m = date_today - relativedelta(months=6)

# Crear date_forecas como seis meses después de date_today
date_forecas = date_today + relativedelta(months=6)

# Mostrar las fechas
print("date_today:", date_today.strftime("%Y-%m-%d"))

print("date_last_6m:", date_last_6m.strftime("%Y-%m-%d"))

print("date_forecas:", date_forecas.strftime("%Y-%m-%d"))

#--------------------------------FUNCIONES--------------------------------#

def transform_MIDAS(df, pv, join_pv = False):
    # --- Define columns to extract ---
    colNames = ["FECHA", "Y_PREDICCION", "NOMBRESUBDIRECCION", "NOMBREGRUPOESTADISTICO3", "MODEL"]
    colNames2 = ["FECHA", "Y_PRONOSTICOS", "NOMBRESUBDIRECCION", "NOMBREGRUPOESTADISTICO3", "MODEL"]
    colNames3 = ["FECHA", "Y_HIST", "NOMBRESUBDIRECCION", "NOMBREGRUPOESTADISTICO3", "MODEL"]

    # --- First Filter: Y_PREDICCION --- Filtro de valores nulos? por que habria valores nulos? 
    df1 = df[df["Y_PREDICCION"].notna()][colNames]

    # --- Second Filter: Y_PRONOSTICOS --- Filtro de valores nulos? por que habria valores nulos?
    df2 = df[df["Y_PRONOSTICOS"].notna()][colNames2]

    df2.columns = colNames  # Rename to match df1

    # --- Combine both filtered datasets ---
    combined_df = pd.concat([df1, df2], ignore_index=True)

    # --- Reorder columns: move Y_PREDICCION next to MODEL ---
    cols = combined_df.columns.tolist()

    y_pred_idx = cols.index("Y_PREDICCION")

    model_idx = cols.index("MODEL")

    cols.insert(model_idx + 1, cols.pop(y_pred_idx))

    combined_df = combined_df[cols]

    combined_df["Y_PREDICCION"] = combined_df["Y_PREDICCION"].astype(float)

    #display(combined_df.FECHA.value_counts().sort_index())
    res = pd.pivot_table(combined_df, index=["FECHA", "NOMBRESUBDIRECCION", "NOMBREGRUPOESTADISTICO3"], columns= "MODEL", values= "Y_PREDICCION", aggfunc= "mean" ).reset_index()

    res["MAX_MODEL"] = True

    res = res.rename(columns = {"NOMBREGRUPOESTADISTICO3": "GE3", "NOMBRESUBDIRECCION": "NombreSubdireccion"})

    res["MAX_MODEL"] = res["MAX_MODEL"].fillna(False)
    
    res["FECHA"] = pd.to_datetime(res["FECHA"])
    
    if join_pv == True:

        out = pv.merge(res, how = "left", on = ["FECHA", "NombreSubdireccion", "GE3" ] ,validate = "1:1")
        
        out["MAX_MODEL"] = out["MAX_MODEL"].fillna(False)
        
        return out
    else:

        return res
    

def load_pv(query_str):

    client = bigquery.Client(project="demanda-prj-dev")

    consulta = query_str

    query = client.query(consulta)

    pv = query.to_dataframe(create_bqstorage_client=True)

    pv.sort_index(inplace=True, ascending=True)

    pv["PVO"] = pv["PVO"].astype(float)

    pv["PV"] = pv["PV"].astype(float)

    pv["FACT"] = pv["FACT"].astype(float).fillna(0)

    #pv["MAX"] = pv["MAX"].astype(float)

    pv["id"] = pv['NombreGrupo'] + "_" + pv['NombreDireccion'] + "_" + pv['NombreSubdireccion'] + "_" + pv['GE1'] + "_" + pv['GE2'] + "_" + pv['GE3']

    # Obtener todos los valores únicos de cada columna clave
    fechas = pv['FECHA'].unique()
    id = pv['id'].unique()

    # Generar todas las combinaciones posibles
    combinaciones = pd.DataFrame(list(itertools.product(fechas, id)), columns=['FECHA',"id"])
    for i, col  in enumerate(["NombreGrupo", "NombreDireccion", "NombreSubdireccion", "GE1", "GE2", "GE3"]):
        print(i, col)
        combinaciones[col] = combinaciones["id"].str.split("_").apply(lambda x:x[i])

    # Unir con el DataFrame original
    df_completo = pd.merge(combinaciones, pv, on=["FECHA","id","NombreGrupo", "NombreDireccion", "NombreSubdireccion", "GE1", "GE2", "GE3"], how='left')

    df_completo["FACT"] = df_completo["FACT"].fillna(0)

    df_completo["valid_ton"] = df_completo.PVO.fillna(0) + df_completo.FACT.fillna(0)

    df_completo["FECHA"] = pd.to_datetime(df_completo["FECHA"])

    pv = df_completo.copy()
    
    return pv

date_today: 2025-09-01
date_last_6m: 2025-03-01
date_forecas: 2026-03-01


In [6]:
# declaramos las credenciales al entorno de desarollo
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "..\\..\\bd\\0_docs\\datahub-deacero-adojeda.json"

query_str = """
SELECT
FORMAT_DATE('%Y-%m-%d', PARSE_DATE('%B.%Y', mes_anio)) AS FECHA,
nom_grupo AS NombreGrupo,
nom_direccion AS NombreDireccion,
nom_subdireccion AS NombreSubdireccion,
nom_grupo_estadistico1 AS GE1,
nom_grupo_estadistico2 AS GE2,
nom_grupo_estadistico3 AS GE3,
sum(toneladas_pvo) AS PVO,
sum(toneladas_facturadas) AS FACT,
#sum(toneladas_mejor_pronostico) AS MAX,
sum(toneladas_plan_ventas) AS PV
FROM `datahub-deacero.mart_comercial.comercial` 
WHERE 
EXTRACT(YEAR FROM fecha) >= 2025
AND
EXTRACT(YEAR FROM fecha) < 2026
AND
nom_direccion NOT IN  ('FILIALES') --> lo quité de la consulta duplica SUBDIRG3
AND
nom_subdireccion NOT IN  ('SERVICIOS LOGISTICOS DAL USA') --> lo quité de la consulta duplica SUBDIRG3
-- Descartamos los siguientes campos
#AND
#nom_grupo NOT IN ('EMPRESAS RELACIONADAS','TRASPASOS E INTEREMPRESAS')
#AND
#nom_direccion NOT IN ('EXPORTACIÓN ALAMBRES')
#AND
#nom_subdireccion NOT IN ('EXCEDENTES ACEROS','EXCEDENTES ALAMBRES','RESTO ALAMBRES','RESTO INGETEK')
#AND
#nom_grupo_estadistico1 NOT IN ('CHATARRA','DEACERO POWER','INTERNAS PRODUCCION ','LOGÍSTICA','PALANQUILLA','SEGUNDAS')
#AND
#nom_grupo_estadistico2 NOT IN ('SEGUNDAS / EXCEDENTES','CHQ')
#AND
#nom_grupo_estadistico3 NOT IN ('DESPERDICIO','ALAMBRON SEGUNDAS','ALAMBRON EXCEDENTES','ALAMBRON TERCEROS','ALAMBRON OTROS','DERECHO DE VIA','PILOTES','PISO','POLIZAS')
-- Necesitamos que la informacion no sea nula
AND nom_grupo IS NOT NULL
AND nom_direccion IS NOT NULL
AND nom_subdireccion IS NOT NULL
AND nom_grupo_estadistico1 IS NOT NULL
AND nom_grupo_estadistico2 IS NOT NULL
AND nom_grupo_estadistico3 IS NOT NULL
AND  mes_anio IS NOT NULL
GROUP BY 1,2,3,4,5,6,7
ORDER BY 1,2,3,4,5,6,7
"""

pv = load_pv(query_str)

display(pv.FECHA.value_counts())

pv.sample(5)

c:\Users\adojeda\OneDrive - deacero.com\dev\0_envs\.bigquery\Lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


0 NombreGrupo
1 NombreDireccion
2 NombreSubdireccion
3 GE1
4 GE2
5 GE3


FECHA
2025-01-01    2181
2025-02-01    2181
2025-03-01    2181
2025-04-01    2181
2025-05-01    2181
2025-06-01    2181
2025-07-01    2181
2025-08-01    2181
2025-09-01    2181
2025-10-01    2181
2025-11-01    2181
2025-12-01    2181
Name: count, dtype: int64

,FECHA,id,NombreGrupo,NombreDireccion,NombreSubdireccion,GE1,GE2,GE3,PVO,FACT,PV,valid_ton
17882,2025-09-01,ALAMBRES_ALAMBRES_INDUSTRIAS_MALLAS Y ALAMBRES...,ALAMBRES,ALAMBRES,INDUSTRIAS,MALLAS Y ALAMBRES,CERCOS ESPECIALIZADOS,REJA ORNAMENTAL,NaN,0.000,NaN,0.00000
16637,2025-08-01,TRASPASOS E INTEREMPRESAS_EXPORTACION FILIALES...,TRASPASOS E INTEREMPRESAS,EXPORTACION FILIALES,FILIALES,MALLAS Y ALAMBRES,CERCOS,CICLONICA GALVANIZADA,381.51473,70.036,381.51473,451.55073
22878,2025-11-01,DEACERO SOLUTIONS_INGETEK_DEFENSA_PERFILES_RED...,DEACERO SOLUTIONS,INGETEK,DEFENSA,PERFILES,REDONDOS Y CUADRADOS,REDONDOS COMERCIALES,NaN,0.000,NaN,0.00000
3817,2025-02-01,USA_EUA - CANADA_AGRICULTURAL DISTRIBUTION_MAL...,USA,EUA - CANADA,AGRICULTURAL DISTRIBUTION,MALLAS Y ALAMBRES,CERCOS ESPECIALIZADOS,PANEL,837.97020,1465.469,837.97020,2303.43920
1122,2025-01-01,EMPRESAS RELACIONADAS_ABAST. EXPORT_VENTAS CAR...,EMPRESAS RELACIONADAS,ABAST. EXPORT,VENTAS CARGA METALICA,CHATARRA,CHATARRA NO FERROSA,COBRE,NaN,146.051,NaN,146.05100


In [7]:
LIST_DIR ="../../bd/Facturacion BQ/MIDAS"

list_df = []
for path in  os.listdir(LIST_DIR):
    print(path)
    df =pd.read_excel(LIST_DIR + "/" + path)
    list_df.append(df)
    del df

base = pd.concat(list_df)

del list_df

base_t = transform_MIDAS(base, pv, join_pv= True)


outputs_BayesianRidge_FORECAST_NombreSubdireccion_ToneladasFacturadas_20250908_190038.xlsx
outputs_GradientBoostingRegressor_FORECAST_NombreSubdireccion_ToneladasFacturadas_20250908_194916.xlsx
outputs_KNeighborsRegressor_FORECAST_NombreSubdireccion_ToneladasFacturadas_20250908_193625.xlsx
outputs_LGBMRegressor_FORECAST_NombreSubdireccion_ToneladasFacturadas_20250908_191256.xlsx
outputs_RandomForestRegressor_FORECAST_NombreSubdireccion_ToneladasFacturadas_20250908_180654.xlsx
outputs_Ridge_FORECAST_NombreSubdireccion_ToneladasFacturadas_20250908_192506.xlsx
outputs_SVR_FORECAST_NombreSubdireccion_ToneladasFacturadas_20250908_184650.xlsx
outputs_XGBRegressor_FORECAST_NombreSubdireccion_ToneladasFacturadas_20250908_183302.xlsx


C:\Users\adojeda\AppData\Local\Temp\ipykernel_7772\1117820063.py:80: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out["MAX_MODEL"] = out["MAX_MODEL"].fillna(False)


In [20]:
#base.groupby("FECHA")[].isna()
base.groupby("FECHA")[['Y_HIST', 'Y_TEST', 'Y_PREDICCION', 'Y_PRONOSTICOS']].sum().round(0)

,Y_HIST,Y_TEST,Y_PREDICCION,Y_PRONOSTICOS
FECHA,,,,
2024-07-01,1381520.0,0.0,0.0,0.0
2024-08-01,1400468.0,0.0,0.0,0.0
2024-09-01,1332355.0,0.0,0.0,0.0
2024-10-01,1400518.0,0.0,0.0,0.0
2024-11-01,1350270.0,0.0,0.0,0.0
2024-12-01,1201723.0,0.0,0.0,0.0
2025-01-01,1581523.0,1581523.0,1450436.0,0.0
2025-02-01,1454739.0,1454739.0,1461348.0,0.0
2025-03-01,1539974.0,1539974.0,1465086.0,0.0
